In [1]:
import pandas as pd
import plotly.plotly as py
import plotly.offline as offline
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import numpy as np

### Read datasets

In [2]:
flow = pd.read_csv("/Users/sergiers/Desktop/CLASE/BDMA/TUe/Capita_selecta/waterschap-aa-en-maas_sewage/sewer_data/PUMPdata_pump_aarle-rixtel_flow-level_1.5years/flow/carpeta sin título/4150/GBS_AR.RG4150_Instellingen_FT510_AM_Debietmeting.Q.csv", delimiter=";", decimal=",", dtype={'level': float, 'flow': float})
level = pd.read_csv("/Users/sergiers/Desktop/CLASE/BDMA/TUe/Capita_selecta/waterschap-aa-en-maas_sewage/sewer_data/PUMPdata_pump_aarle-rixtel_flow-level_1.5years/flow/carpeta sin título/4150/GBS_AR.RG4160_Instellingen_LT610_Niveaumeting.Q.csv", delimiter=";", decimal=",", dtype={'level': float, 'flow': float})
rain = pd.read_csv("/Users/sergiers/Desktop/CLASE/BDMA/TUe/Capita_selecta/waterschap-aa-en-maas_sewage/sewer_data/PUMPdata_pump_aarle-rixtel_flow-level_1.5years/flow/carpeta sin título/4150/4150-Boerdonk.csv", delimiter=";", decimal=",", dtype={'level': float, 'flow': float})

#a = pd.read_csv("/Users/sergiers/Desktop/CLASE/BDMA/TUe/Capita_selecta/waterschap-aa-en-maas_sewage/sewer_data/PUMPdata_pump_aarle-rixtel_flow-level_1.5years/flow/carpeta sin título/4160/GBS_AR.RG4160_Instellingen_FT510_Debietmeting.Q_1-1-2017.csv", delimiter=";", decimal=",", dtype={'Value': float})
#b = pd.read_csv("/Users/sergiers/Desktop/CLASE/BDMA/TUe/Capita_selecta/waterschap-aa-en-maas_sewage/sewer_data/PUMPdata_pump_aarle-rixtel_flow-level_1.5years/flow/carpeta sin título/4160/GBS_AR.RG4160_Instellingen_FT510_Debietmeting.Q_7-8-2017.csv", delimiter=";", decimal=",", dtype={'Value': float})
#flow = a.append(b)

In [3]:
pumpNumber = "4150"

In [4]:
#Preprocess:
#Flow:
flow["TimeStamp"] = pd.to_datetime(flow['TimeStamp'], format='%d-%m-%Y %H:%M:%S')
flow.index = flow["TimeStamp"]
flow["flow"] = flow["Value"]
flow = flow[["flow"]]

#level:
level["TimeStamp"] = pd.to_datetime(level['TimeStamp'], format='%d-%m-%Y %H:%M:%S')
level.index = level["TimeStamp"]
level["level"] = level["Value"]
level = level[["level"]]

#Rain:
rain["Eind"] = pd.to_datetime(rain['Eind'], format='%d.%m.%y %H:%M')
rain.index = rain["Eind"]
rain = rain[["VOLUME", "WET"]]

### Resample data:

In [5]:
level = level.resample('1T').mean()
flow = flow.resample('1T').mean()
rain = rain.resample('1T').mean()

### Fill nan and repair

In [6]:
#fill nan
flow = flow.fillna(0)
#flow= flow.interpolate(method='pchip')

#remove level 0
level = level[level["level"] != 0]
level= level.interpolate(method='pchip')

#Fill nan with previous values
rain = rain.fillna(method='ffill')

### Shot counter:

In [7]:
#threashold = 100
#flow["working"] = np.where(flow["flow"]<threashold, 0, 1)

### Join Datasets

In [8]:
#Inner join Datasets
a = flow.join(level, how='inner')
dataset = a.join(rain, how='inner')

### Day of the week and week of year

In [9]:
dataset['WeekNumber'] = dataset.index.week
dataset['DayOfWeek'] = dataset.index.weekday

### Get means:

In [10]:
#onmean = dataset.loc[dataset['working'] == 1]["flow"].mean()
#offmean = dataset.loc[dataset['working'] == 0]["flow"].mean()

### Create train dataset:

In [11]:
#dataset["working"] = np.where(dataset["working"]==0, offmean, onmean)

In [12]:
#dataset = dataset[["working", "level", "VOLUME", "WET", "WeekNumber", "DayOfWeek"]]

In [13]:
#dataset = dataset.rename(columns={'working': 'flow'})

### Create DS with/without

In [14]:
datadry = dataset[dataset["WET"] != 1]
datawet = dataset[dataset["WET"] != 0]

In [15]:
datadry = dataset[["flow", "level", "WeekNumber", "DayOfWeek"]]

In [16]:
#test = dataset.loc['2017-09-01 00:00':'2017-09-06 00:00']

In [17]:
datadry.to_csv(pumpNumber+"setdry.csv", sep=',')
datawet.to_csv(pumpNumber+"setwet.csv", sep=',')

### Fix Dataset (if necessary)

In [18]:
# Remove NaN
#dataset = dataset.dropna()

In [19]:
#Fill NaN with Mean?
#dataset = dataset.fillna(flow.mean())

In [20]:
# Remove Zeros?
#dataset = dataset[dataset["flow"] != 0]
#dataset = dataset[dataset["level"] != 0]